# DCA Optimization Results

This notebook demonstrates the DCA (Dollar Cost Averaging) optimizer by finding optimal allocation ratios across different assets.

## Overview

The DCA optimizer performs a grid search across:
- Asset allocation ratios (e.g., 60/40, 70/30, 80/20)
- Investment durations (e.g., 5 years, 10 years)
- Purchase intervals (monthly, quarterly)

For each combination, it calculates:
- CAGR (Compound Annual Growth Rate)
- Sharpe Ratio
- Maximum Drawdown
- Standard Deviation

## Portfolio Demonstrated

- **SPY/TLT**: Classic 60/40 stock/bond portfolio
- **UPRO/TMF**: Leveraged version (3x S&P 500 / 3x 20-year Treasury)
- **SPY/TQQQ**: Growth-focused mix

In [ ]:
# Setup
import warnings

warnings.filterwarnings("ignore")

# Set environment
import os

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

os.environ["DYNACONF_ENV"] = "development"

from config import logger

logger.info("Notebook initialized")

## 1. Load Price Data

Load historical price data for the assets we'll optimize.

In [ ]:
from finbot.utils.data_collection_utils.yfinance.get_history import get_history

# Load asset price histories
assets = ["SPY", "TLT", "UPRO", "TMF", "TQQQ"]
price_data = {}

for asset in assets:
    try:
        price_data[asset] = get_history(asset, adjust_price=True)
        print(f"✓ Loaded {asset}: {len(price_data[asset])} days")
    except Exception as e:
        print(f"✗ Error loading {asset}: {e}")

print(f"\nSuccessfully loaded {len(price_data)} assets")

## 2. Run DCA Optimizer

Run the optimizer to find optimal allocation ratios.

In [ ]:
from finbot.services.optimization.dca_optimizer import DCAOptimizer

# Example: Optimize SPY/TLT portfolio
optimizer = DCAOptimizer(
    asset1_data=price_data["SPY"], asset2_data=price_data["TLT"], asset1_name="SPY", asset2_name="TLT"
)

# Run optimization
# Ratios to test: 50/50, 60/40, 70/30, 80/20, 90/10
# Duration: 10 years
# Interval: Monthly
results = optimizer.optimize(ratios=np.arange(0.5, 1.0, 0.1), duration_years=10, interval="monthly")

print("\nOptimization Results (SPY/TLT):")
print("=" * 80)
print(results.head(10))

## 3. Visualize Optimization Surface

Create 3D surface plots showing how performance metrics vary with allocation ratios.

In [ ]:
# Create subplots for different metrics
fig = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=["CAGR vs Ratio", "Sharpe Ratio vs Ratio", "Max Drawdown vs Ratio", "Std Dev vs Ratio"],
    vertical_spacing=0.12,
    horizontal_spacing=0.10,
)

# CAGR
fig.add_trace(go.Scatter(x=results["ratio"], y=results["cagr"], mode="lines+markers", name="CAGR"), row=1, col=1)

# Sharpe Ratio
fig.add_trace(go.Scatter(x=results["ratio"], y=results["sharpe"], mode="lines+markers", name="Sharpe"), row=1, col=2)

# Max Drawdown (negative values, so we multiply by -1 for display)
fig.add_trace(
    go.Scatter(x=results["ratio"], y=results["max_drawdown"] * -1, mode="lines+markers", name="Max DD"), row=2, col=1
)

# Standard Deviation
fig.add_trace(go.Scatter(x=results["ratio"], y=results["std_dev"], mode="lines+markers", name="Std Dev"), row=2, col=2)

fig.update_layout(height=800, title_text="DCA Optimization Results: SPY/TLT Portfolio", showlegend=False)

fig.update_xaxes(title_text="SPY Ratio", row=1, col=1)
fig.update_xaxes(title_text="SPY Ratio", row=1, col=2)
fig.update_xaxes(title_text="SPY Ratio", row=2, col=1)
fig.update_xaxes(title_text="SPY Ratio", row=2, col=2)

fig.update_yaxes(title_text="CAGR", row=1, col=1)
fig.update_yaxes(title_text="Sharpe Ratio", row=1, col=2)
fig.update_yaxes(title_text="Max Drawdown (abs)", row=2, col=1)
fig.update_yaxes(title_text="Std Dev", row=2, col=2)

fig.show()

## 4. Compare Multiple Portfolios

Run optimization for different portfolio combinations and compare optimal ratios.

In [ ]:
# Compare three portfolios
portfolio_pairs = [("SPY", "TLT", "Conservative"), ("UPRO", "TMF", "Leveraged"), ("SPY", "TQQQ", "Growth")]

comparison_results = []

for asset1, asset2, name in portfolio_pairs:
    if asset1 in price_data and asset2 in price_data:
        opt = DCAOptimizer(
            asset1_data=price_data[asset1], asset2_data=price_data[asset2], asset1_name=asset1, asset2_name=asset2
        )

        res = opt.optimize(ratios=np.arange(0.5, 1.0, 0.1), duration_years=10, interval="monthly")

        # Find optimal ratio by Sharpe
        best_idx = res["sharpe"].idxmax()
        best = res.loc[best_idx]

        comparison_results.append(
            {
                "Portfolio": name,
                "Assets": f"{asset1}/{asset2}",
                "Optimal Ratio": f"{best['ratio']:.0%}/{1 - best['ratio']:.0%}",
                "CAGR": best["cagr"],
                "Sharpe": best["sharpe"],
                "Max DD": best["max_drawdown"],
                "Std Dev": best["std_dev"],
            }
        )

comparison_df = pd.DataFrame(comparison_results)
print("\nPortfolio Comparison (Optimized by Sharpe Ratio):")
print("=" * 100)
print(comparison_df.to_string(index=False))

## 5. Sensitivity Analysis

Test how optimal ratios change with different time horizons.

In [ ]:
# Test SPY/TLT with different durations
durations = [5, 10, 15, 20]
sensitivity_results = []

for duration in durations:
    opt = DCAOptimizer(
        asset1_data=price_data["SPY"], asset2_data=price_data["TLT"], asset1_name="SPY", asset2_name="TLT"
    )

    res = opt.optimize(ratios=np.arange(0.5, 1.0, 0.05), duration_years=duration, interval="monthly")

    best_idx = res["sharpe"].idxmax()
    best = res.loc[best_idx]

    sensitivity_results.append(
        {
            "Duration (years)": duration,
            "Optimal Ratio": f"{best['ratio']:.0%}",
            "CAGR": best["cagr"],
            "Sharpe": best["sharpe"],
        }
    )

sensitivity_df = pd.DataFrame(sensitivity_results)
print("\nSensitivity to Investment Duration (SPY/TLT):")
print("=" * 80)
print(sensitivity_df.to_string(index=False))

## Key Findings

The DCA optimizer reveals:

1. **Classic 60/40 Portfolio (SPY/TLT)**: Traditional wisdom of 60% stocks / 40% bonds often emerges as optimal for risk-adjusted returns

2. **Leveraged Portfolios (UPRO/TMF)**: Require careful rebalancing and can achieve higher Sharpe ratios with lower allocation ratios due to leverage amplification

3. **Growth Portfolios (SPY/TQQQ)**: Higher equity ratios maximize CAGR but at the cost of increased volatility and drawdowns

4. **Time Horizon Effect**: Longer investment horizons tend to favor higher equity allocations as short-term volatility smooths out

5. **Sharpe Optimization**: Optimizing for Sharpe ratio (risk-adjusted returns) often produces more conservative allocations than optimizing for pure CAGR

## Next Steps

- Test with different rebalancing frequencies (quarterly, annually)
- Include transaction costs and tax considerations
- Extend to 3+ asset portfolios
- Compare DCA vs lump-sum investment strategies
- Run Monte Carlo simulations on optimal portfolios